In [ ]:
"""
Model Result Processing
Author: Liam Megraw, RIT Envirionmental Science Technician
Date last edited: 11/15/2022
ESRI ArcGIS Pro Version 2.5.2

Description:


Output:


How to Use:

"""

In [1]:
# Set environment -----
import arcpy
import os

arcpy.env.workspace = "C:\\Users\\ltm4654\\Documents\\ArcGIS\\Projects\\Final_Deployment\\Final_Deployment.gdb"
arcpy.env.overwriteOutput = True

In [4]:
# Dictionary of land covers
cover_classes = {
    11: 'Water',
    21: 'Dev_OS',
    22: 'Dev_LI',
    23: 'Dev_MI',
    24: 'Dev_HI',
    31: 'Barren',
    41: 'Forest1',
    42: 'Forest2',
    43: 'Forest3',
    52: 'Shrub',
    71: 'Herb',
    81: 'PH',
    82: 'Crop',
    90: 'WW',
    95: 'EHW'
}

gridcodes = list(cover_classes.keys())
cover_rasters = list()

In [33]:
start = time.time()
from arcpy.sa import *
# Iterate over cover types
for gridcode in gridcodes:
    print(cover_classes[gridcode], gridcode)
    # Make raster of just that cover type
    print("***Masking")
    perCoverMask = arcpy.sa.Mask('NLCD_NYS_2016_18N', "", [gridcode, gridcode])
    # Create raster summing all grid codes of this cover type within a neighborhood
    print("***Focal stats")
    perCoverNeighborhood = FocalStatistics(perCoverMask, NbrCircle(3, "CELL"), "SUM")
    # Map algebra to convert sum to count by dividing the gridcode sum by the gridcode
    print("***Creating count raster")
    perCoverCount = Divide(perCoverNeighborhood, gridcode)
    countRas = cover_classes[gridcode]
    perCoverCount.save(countRas)
    # Add raster to list
    cover_rasters.append(countRas)
    del perCoverMask, perCoverNeighborhood, perCoverCount, countRas

# Extract raster values to points
ExtractMultiValuesToPoints('pred_finalDeployment_allSpecies', cover_rasters)

end = time.time()
print("Runtime: ",(end-start)," s")

Water 11
***Masking
***Focal stats
***Creating count raster
Dev_OS 21
***Masking
***Focal stats
***Creating count raster
Dev_LI 22
***Masking
***Focal stats
***Creating count raster
Dev_MI 23
***Masking
***Focal stats
***Creating count raster
Dev_HI 24
***Masking
***Focal stats
***Creating count raster
Barren 31
***Masking
***Focal stats
***Creating count raster
Forest1 41
***Masking
***Focal stats
***Creating count raster
Forest2 42
***Masking
***Focal stats
***Creating count raster
Forest3 43
***Masking
***Focal stats
***Creating count raster
Shrub 52
***Masking
***Focal stats
***Creating count raster
Herb 71
***Masking
***Focal stats
***Creating count raster
PH 81
***Masking
***Focal stats
***Creating count raster
Crop 82
***Masking
***Focal stats
***Creating count raster
WW 90
***Masking
***Focal stats
***Creating count raster
EHW 95
***Masking
***Focal stats
***Creating count raster
Runtime:  2545.607613325119  s


In [12]:
# Check for cell count errors from the joining process
import pandas as pd
# Registers the spatial functionality to let you perform spatial operations with pandas objects
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# Create and view spatial dataframe
sdf = pd.DataFrame.spatial.from_featureclass('pred_finalDeployment_allSpecies')
sdf.head()

,objectid,lat,long,date,phra,knot,wild_parsnip,purple_loosestrife,tree_of_heaven_with_seeds,water,dev_os,dev_li,dev_mi,dev_hi,barren,forest1,forest2,forest3,shrub,herb,ph,crop,ww,ehw,SHAPE
0,1,41.403096,-74.322725,2018-08,0.00,0.01,0,0.01,0.22,0.0,2.0,4.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{""x"": 556611.7395673255, ""y"": 4583726.47319956..."
1,2,41.395831,-74.342401,2017-09,0.00,0.01,0,0.01,0.01,0.0,9.0,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,"{""x"": 554973.1884894976, ""y"": 4582907.25526065..."
2,3,41.391915,-74.333993,2017-09,1.00,0.02,0.01,0.01,0.01,0.0,1.0,10.0,6.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,"{""x"": 555679.4143230421, ""y"": 4582477.87754025..."
3,4,41.397663,-74.374896,2016-11,0.75,0.01,0,0.01,0.01,0.0,2.0,3.0,4.0,2.0,5.0,0.0,0.0,0.0,0.0,2.0,11.0,0.0,0.0,0.0,"{""x"": 552255.2355604023, ""y"": 4583090.53226935..."
4,5,41.382348,-74.304529,2016-11,0.15,0.01,0,0.01,0.01,0.0,5.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,"{""x"": 558151.1915404452, ""y"": 4581435.12019072..."


In [ ]:
# Evaluate fields to see if all numbers are in the expected range
covers = list(cover_classes.values())
e = list()
for field in covers:
    # Fill in nulls with zero
    sdf[field].fillna(0, inplace = True)
    # Get maximum value in field
    m = max(sdf[field])
    # Evaluate if max is too large (29 is the max number of cells in a 3-cell radius)
    if m > 29:
        e.append("Error")
        print("Field '"+field+"' exceeds the maximum cell count")
    else:
        pass
if "Error" in e:
    pass
else:
    print("No errors detected")

In [15]:
print(type(sdf['objectid'][0]))

<class 'numpy.int64'>


In [20]:
arcpy.env.overwriteOutput = True
# Re-export as a new fc with zeroes instead of null values
sdf.loc[0:4,:].spatial.to_featureclass(location=r'C:\Users\ltm4654\Documents\ArcGIS\Projects\Final_Deployment\Final_Deployment.gdb\pred_FD_test');

'C:\\Users\\ltm4654\\Documents\\ArcGIS\\Projects\\Final_Deployment\\Final_Deployment.shp'